# [HuggingFace NLP Course](https://huggingface.co/learn/nlp-course)

This notebook documents some of my experiments done while completing this HuggingFace course.

## [2. Using Transformers](https://huggingface.co/learn/nlp-course/chapter2)

### [Behind the pipeline](https://huggingface.co/learn/nlp-course/chapter2/2)

> "All 🤗 Transformers models output the logits, as the loss function for training will generally fuse the last activation function, such as SoftMax, with the actual loss function, such as cross entropy):"

Note: AutoModel means "an object that returns the correct architecture based on the checkpoint."









In [3]:
from transformers import pipeline

print('\nfill-mask:')
unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])


print('\nsentiment analysis:')
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)


fill-mask:


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']

sentiment analysis:


[{'label': 'POSITIVE', 'score': 0.9598050713539124},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [4]:
from transformers import AutoTokenizer, AutoModel
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

## encode input sentences to input_ids
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
    "the dog is sitting on the ground wagging its tail",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
# note the attention mask indicates where padding was applied in each sentence
print("tokenizer inputs:")
print(inputs)
print(inputs.input_ids.shape)
print(f"key: {tokenizer.convert_ids_to_tokens(inputs.input_ids[0])}")


## get the feature vectors for each input sequence
model = AutoModel.from_pretrained(checkpoint)
outputs = model(**inputs)
print("\feature vector noutputs:")
print(outputs.last_hidden_state.shape)


## lets classify the sentiment of each sentence
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# passes the input_ids and corresponding attention_mask to the model
sentiment_outputs = model(**inputs)
print("\nsentiment outputs:")
print(sentiment_outputs.logits.shape)
print("sentence 0 raw logits:")
print(sentiment_outputs.logits[0, :])

print("\npreds:")
preds = torch.nn.functional.softmax(sentiment_outputs.logits, dim=-1)
print(preds)

# print final sentiment predictions
key = model.config.id2label 
for i in range(len(raw_inputs)):
    print(f"'{raw_inputs[i]}' => {key[0]}: {preds[i][0]:.3f}, {key[1]}: {preds[i][1]:.3f}")

tokenizer inputs:
{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  1996,  3899,  2003,  3564,  2006,  1996,  2598, 11333, 12588,
          2049,  5725,   102,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}
torch.Size([3, 16])
key: ['[CLS]', 'i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.', '[SEP]']
eature vector noutputs:
torch.Size([3, 16, 768])

sentiment outputs:
torch.Size([3, 2])
sentence 0 raw logits:
tensor([-1.5607,  1.6123], grad_fn=<SliceBackward0>)

preds:
tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5

In [3]:
# equivalent to the above!
classifier = pipeline("sentiment-analysis", model=checkpoint, tokenizer=checkpoint)
# can be loaded equivalently with:
#classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(raw_inputs)

[{'label': 'POSITIVE', 'score': 0.9598050713539124},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455},
 {'label': 'NEGATIVE', 'score': 0.9941940903663635}]

### [Models](https://huggingface.co/learn/nlp-course/chapter2/3)
> How to load and instantiate models

In [5]:
from transformers import BertConfig, BertModel

# config is like a recipe for how to build the model (e.g. define feature size, num layers etc)
config = BertConfig()
print("default BertConfig:")
print(config)

# build model from config
model = BertModel(config)
print("\nmodel architecture:")
print(model)

# save a model to disk
# my-bert-model/
# ├── config.json
# └── model.safetensors
model.save_pretrained("/tmp/models/my-bert-model")
# now load from disk
model = model.from_pretrained("/tmp/models/my-bert-model", local_files_only=True)

# loading from pretrained https://huggingface.co/google-bert/bert-base-cased
model = BertModel.from_pretrained("bert-base-uncased")


default BertConfig:
BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.37.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}


model architecture:
BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(


### [Tokenzier](https://huggingface.co/learn/nlp-course/chapter2/4)

Interesting mention of alternative techniques such as **"SentencePiece or Unigram, as used in several multilingual models"**.

Encoding has two steps:
1. Tokenization (split input text into tokens) e.g. by subword method
2. Convert tokens into tensors (using the tokenizer's vocabulary)

In [23]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)
print(f"tokens:\n", tokens)

# convert token strings to input IDs
ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"ids:\n", ids)
print(f"decoded: '{tokenizer.decode(ids)}'")


# directly reading the vocabulary (dictionary)
vocab = tokenizer.get_vocab()
print(f"\nvocab type: {type(vocab)}, len: {len(vocab)}")
for k, v in vocab.items():
    print(f"example vocab: '{k}' => {v}")
    break

# invert vocabulary to map token ids (int) -> token strings (str)
id_to_token = {id: token for token, id in vocab.items()}
my_decoded = [id_to_token[id] for id in ids]
print("my_decoded:\n", my_decoded)
#print(vocab['##former']) # returns 23763

tokens:
 ['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']
ids:
 [7993, 170, 13809, 23763, 2443, 1110, 3014]
decoded: 'Using a Transformer network is simple'

vocab type: <class 'dict'>, len: 28996
example vocab: 'shy' => 12076
my_decoded:
 ['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


### [Handling Multiple Sequences](https://huggingface.co/learn/nlp-course/chapter2/5)
Elaborates on how to tokenize batches of text, and the need to pad the sequences to match the longest in the batch, and the use of attention masks to tell attention layers to ignore the padding tokens.

References some transformer models specifically designed for handling longer sequence lengths:
* [Longformer](https://huggingface.co/docs/transformers/model_doc/longformer)
* [LED](https://huggingface.co/docs/transformers/model_doc/led)

In [34]:
print(f"pad_token_id = {tokenizer.pad_token_id}, pad_token = '{tokenizer.pad_token}'")

# more info: https://huggingface.co/docs/transformers/pad_truncation
# when tokenizing we can specify a max length and padding strategy
# padding="longest" | "max_length" | "do_not_pad"
# we also request to receive the results as pytorch tensors instead of lists
model_inputs = tokenizer(raw_inputs, max_length=4, truncation=True, return_tensors="pt")
print(model_inputs)

pad_token_id = 0, pad_token = '[PAD]'
{'input_ids': tensor([[ 101, 1045, 1005,  102],
        [ 101, 1045, 5223,  102],
        [ 101, 1996, 3899,  102]]), 'attention_mask': tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]])}


## [3. Fine-Tuning a Pretrained Model](https://huggingface.co/learn/nlp-course/chapter3/1)

### [Processing the Data](https://huggingface.co/learn/nlp-course/chapter3/2)
Note: this section of the coures also starts with a hint of a very manual way  to do the training directly with PyTorch.

In [31]:
import datasets # pip install datasets

# load the MRPC dataset (task) from the GLUE benchmark
# https://huggingface.co/datasets/nyu-mll/glue/viewer/mrpc
raw_datasets = datasets.load_dataset("glue", "mrpc")

In [38]:
# exploring the dataset a bit
print(raw_datasets)
raw_train_dataset = raw_datasets["train"]

print("\ndataset features:")
print(raw_train_dataset.features)

print("\nclass labels:")
class_labels: datasets.features.ClassLabel = raw_train_dataset.features["label"]
print(class_labels.names)
# we can convert labels from int <-> str
print(f"not_equivalent = label {class_labels.str2int('not_equivalent')}, equivalent = label {class_labels.str2int('equivalent')}")
assert class_labels.int2str(class_labels.str2int("not_equivalent")) == "not_equivalent"


print("first training example:")
raw_train_dataset[0]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

dataset features:
{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None), 'idx': Value(dtype='int32', id=None)}

class labels:
['not_equivalent', 'equivalent']
not_equivalent = label 0, equivalent = label 1
first training example:


{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [45]:
# preparing the dataset
from transformers import AutoTokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# jointly tokenize two sentences
# the field token_type_ids is returned (by this particular model) to distinguish the two sentences (by index)
res = tokenizer(raw_train_dataset[0]["sentence1"], raw_train_dataset[0]["sentence2"])
print(res)
print(tokenizer.convert_ids_to_tokens(res["input_ids"]))
#tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])


# now we tokenize all training sentences together
#tokenized_training_dataset = tokenizer(
#    raw_datasets["train"]["sentence1"],
#    raw_datasets["train"]["sentence2"],
#    padding=True,
#    truncation=True,
#)

# ^this is less memory efficient (it returns a dictionary) so instead we do:
def tokenize_function(example):
    # intentionally not padding here yet (each batch should have the minimum padding necessary)
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)
# tokenize all datasets at once
#   Dataset.map() keeps the data as a dataset,  batched=True uses multithreading
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

{'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'am', '##ro', '##zi', 'accused', 'his', 'brother', ',', 'whom', 'he', 'called', '"', 'the', 'witness', '"', ',', 'of', 'deliberately', 'di', '##stor', '##ting', 'his', 'evidence', '.', '[SEP]', 'referring', 'to', 'him', 'as', 'only', '"', 'the', 'witness', '"', ',', 'am', '##ro', '##zi', 'accused', 'his', 'brother', 'of', 'deliberate

Map: 100%|██████████| 1725/1725 [00:00<00:00, 6681.88 examples/s]


### [Fine-tuning a model with the Trainer API](https://huggingface.co/learn/nlp-course/chapter3/3)

In [72]:
import numpy as np
import evaluate # pip install evaluate scikit-learn
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification

# and now we can apply "dynamic padding" (each batch padded to the length of its longest sequence)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    "/tmp/test_trainer",
    # evaluate every epoch
    evaluation_strategy="epoch",
)
trainer = Trainer(
    model, 
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [73]:
trainer.train()







































































































































































































































































































































































































































































































                                                  

                                            
                       


 33%|███▎      | 459/1377 [19:47<11:41,  1.31it/s]






{'eval_loss': 0.39708858728408813, 'eval_accuracy': 0.8406862745098039, 'eval_f1': 0.8869565217391304, 'eval_runtime': 7.7546, 'eval_samples_per_second': 52.614, 'eval_steps_per_second': 6.577, 'epoch': 1.0}











































                                                  

                                            
 33%|███▎      | 459/1377 [19:55<11:41,  1.31it/s]


{'loss': 0.5287, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}






























































































































































































































































































































































































































































                                                  

                                            
                       


 33%|███▎      | 459/1377 [21:27<11:41,  1.31it/s]






...



















































































                                                  

                                             
 33%|███▎      | 459/1377 [21:45<11:41,  1.31it/s]


{'loss': 0.313, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}





















































































































































































































































































































































































































                                                  

                                             
                       


 33%|███▎      | 459/1377 [23:12<11:41,  1.31it/s]




                                                  

                                             
100%|██████████| 1377/1377 [05:11<00:00,  4.42it/s]

{'eval_loss': 0.6169558763504028, 'eval_accuracy': 0.8627450980392157, 'eval_f1': 0.903448275862069, 'eval_runtime': 2.4122, 'eval_samples_per_second': 169.142, 'eval_steps_per_second': 21.143, 'epoch': 3.0}
{'train_runtime': 311.848, 'train_samples_per_second': 35.286, 'train_steps_per_second': 4.416, 'train_loss': 0.3521837554466231, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3521837554466231, metrics={'train_runtime': 311.848, 'train_samples_per_second': 35.286, 'train_steps_per_second': 4.416, 'train_loss': 0.3521837554466231, 'epoch': 3.0})

### [A full training](https://huggingface.co/learn/nlp-course/chapter3/4)